In [1]:
import lionagi as li
import asyncio

from lionagi import CallDecorator as cd

## Sync only

##### 1. Cache

In [2]:
import time

@cd.cache
def square_data(x):
    time.sleep(1)
    return x**2

result, elapse = await li.tcall(input_=10_000, func=square_data, sleep=0, include_timing=True)
print(f"result: {result:_}, elapse: {elapse:.03f} seconds")

result: 100_000_000, elapse: 1.005 seconds


In [3]:
result, elapse = await li.tcall(input_=10_000, func=square_data, sleep=0, include_timing=True)
print(f"elapse: {elapse:.03f} seconds")

elapse: 0.000 seconds


#### 2. Filter

In [4]:
@cd.filter(predicate=lambda x: x<10)
def square_data(x):
    return [0, x**2]
    
square_data(5)

[0]

## Sync and Async

#### 1. Throttle 

In [5]:
# sync
@cd.throttle(1)
def square_data(x):
    return(x**2)

li.lcall(range(5), square_data)

[0, 1, 4, 9, 16]

In [6]:
# async
@cd.throttle(1)
async def square_data(x):
    return(x**2)

await li.alcall(range(5), await square_data)

[0, 1, 4, 9, 16]

#### 2. pre_post_process (sync and async)

In [7]:
# sync
@cd.pre_post_process(preprocess=lambda x: x+1, postprocess=lambda x: x**2)
def get_5(x):
    return x

get_5(5)

36

In [8]:
# async
@cd.pre_post_process(preprocess=lambda x: x+1, postprocess=lambda x: x**2)
async def get_5(x):
    return x

await get_5(5)

36

## Async only

#### 1. Timeout

In [9]:
@cd.timeout(1)
async def square_data(x):
    await asyncio.sleep(2)
    return x**2

await square_data(5)

TimeoutError: 

#### 2. Retry

In [10]:
@cd.retry(2)
async def square_data(x):
    print("trying")
    raise Exception("error")
    
await square_data(5)

trying
trying


Exception: error

#### 3. Default

In [11]:
@cd.default("tried and failed")
async def square_data(x):
    raise Exception("error")
    
await square_data(5)

'tried and failed'

# To-Do

In [ ]:
"""
7. filter
8. map
9. reduce
10. compose
11. memorize
12. validate
13. max_concurrency
"""